In [ ]:
import scanpy as sc
import pandas as pd
import seaborn as sns
import json
import numpy as np
import sklearn as skl
import sklearn.metrics

import matplotlib.pyplot as plt

## DPI settings

To save space, we use relatively small DPI. Change them to get high quality figures.

In [ ]:
small_dpi = 36
medium_dpi = 45
large_dpi = 50

## Fig. 1 + Table 1
**Fig. 1ab** are illustatives drawn by PowerPoint.

In [ ]:
dump_data = pd.read_csv("Fig-1c.csv")
dump_data = pd.read_csv("Fig-1c.csv")
adata = sc.AnnData(X=dump_data.iloc[:, 4:-1], 
                            obs=dump_data[['label', 'time', 'label_l2']],
                            obsm={'X_umap': dump_data[['UMAP1', 'UMAP2']].values})

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')
sc.pl.umap(adata, color=['label'] + adata.var_names.tolist(), 
           title=['Cell_type', 'Checkpoint', 'Gradient', 'Cell-type specific', 'Shared'], ncols=5, color_map="inferno")

In [ ]:
dump_data = pd.read_csv("table1.csv")

df = pd.DataFrame(index=['SCMER', 'DEG', 'Correlation'], 
                  columns=['checkpointX', 'checkpointY', 'Pro', 'PreX', 'PreY','MatureX', 'MatureY'])

major_cell_types = ['Pro', 'PreX', 'PreY','MatureX', 'MatureY']
checkpoints = ['checkpointX', 'checkpointY']

for i in df.index:
    df.loc[i, checkpoints] = ["%.2f / %.2f" % i 
                          for i in zip(skl.metrics.precision_score(dump_data.Ground_truth_checkpoint, 
                                                                   dump_data[i + '_checkpoint'], zero_division=1, 
                                                                   labels=checkpoints, average=None), 
                                       skl.metrics.recall_score(dump_data.Ground_truth_checkpoint, 
                                                                dump_data[i + '_checkpoint'], zero_division=1,  
                                                                   labels=checkpoints, average=None)[-2:])]
    df.loc[i, major_cell_types] = ["%.2f / %.2f" % i 
                               for i in zip(skl.metrics.precision_score(dump_data.Ground_truth_label, 
                                                                        dump_data[i + '_label'], zero_division=1,  
                                                                   labels=major_cell_types, average=None)[:5], 
                                            skl.metrics.recall_score(dump_data.Ground_truth_label, 
                                                                     dump_data[i + '_label'], zero_division=1,  
                                                                   labels=major_cell_types, average=None)[:5])]
df

## Fig. 2 + Supplementary Figure 1 and 2 + Table 2

**Fig. 2g-i** are generated by [OncoLnc](http://www.oncolnc.org/") online.

For metrics (**Fig. 2c**, and also **Fig. 3i** and **Fig. 4c**), please see **Supplementary Figure 11** below.

**Table 2** is generated by taking the intersection of `SCMER-feature-weights/melanoma-weigths.csv` and `Gene-sets/melanoma-gene-sets.csv`.

In [ ]:
dump_name = 'Fig-2abdef-S1-S2.csv'
dump_features = ['PMEL', 'PDCD1', 'OAS2', 'TOB1', 'HAPLN3', 'CD52', 'TYR', 'TCF7', 'C1orf56', 'VIM', 'MTRNR2L2']

In [ ]:
dump_data = pd.read_csv(dump_name)
adata = sc.AnnData(X=dump_data[dump_features], 
                            obs=dump_data[['Cell_Type', 'Sample']],
                            obsm={'X_original_umap': dump_data[['Original_UMAP1', 'Original_UMAP2']].values, 
                                  'X_scmer_umap': dump_data[['SCMER_UMAP1', 'SCMER_UMAP2']].values})

sc.settings.set_figure_params(dpi=medium_dpi, facecolor='white')
sc.pl.embedding(adata, "original_umap", color=['Cell_Type', 'Sample'], title=['Original', 'Original'])
sc.pl.embedding(adata, "scmer_umap", color=['Cell_Type', 'Sample'], title=['SCMER', 'SCMER'])

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')
sc.pl.embedding(adata, "original_umap", color=dump_features, color_map="inferno")
sc.pl.embedding(adata, "scmer_umap", color=dump_features, color_map="inferno")

In [ ]:
selected_features = set(pd.read_csv("SCMER-feature-weights/melanoma-weigths.csv", index_col=0).index.tolist())
established_features = pd.read_csv("Gene-sets/melanoma-gene-sets.csv", index_col=0)

df = established_features.copy()
for i in df.index:
    df.loc[i] = ",".join(set(df.loc[i, 'genes'].split(", ")).intersection(selected_features))
    
df.style

## Supplementary Figure 3

In [ ]:
dump_data = pd.read_csv("Fig-S3.csv", index_col=0)
sc.settings.set_figure_params(dpi=large_dpi, facecolor='white')

adata = sc.AnnData(X=dump_data.iloc[:, 3:], 
                   obs=dump_data[['Cell_line']],
                   obsm={'X_umap': dump_data[['Original_UMAP1', 'Original_UMAP2']].values})
                                  
sc.pl.umap(adata, color=['Cell_line', 'PRDX1', 'CDC20'], frameon=False, legend_loc=None, 
           palette=sc.pl.palettes.default_20, color_map="inferno")

## Fig. 3 + Supplementary Figure 4, 5, and 6

In [ ]:
dump_features = [['CD79A', 'LYZ', 'FCER2', 'ANKRD28'], 
                 ['JUN', 'FOS', 'CD69', 'ODF2L'], 
                 ['DUSP1'],
                 ['CD7', 'SEPP1'], 
                 ['C1QA', 'C1QB', 'C1QC', 'GNLY'],
                ['GZMA', 'GZMB', 'GZMK', 'GZMH'],
                ['DUSP1', 'DUSP2', 'DUSP4', 'CD69'],
                ['MAF', 'FCER1G', 'ID2', 'ITGA1'],
                ['COL1A2', 'CCL11', 'TFF3', 'CYB561']]
ILC_NK_features = [['SELL', 'GNLY', 'IL7R', 'IRF8'], 
                ['CD160', 'CCL4', 'GZMK', 'GZMA'],
                ['NCR1', 'EOMES']]

In [ ]:
dump_name = 'Fig-3abcdefgh-S4-S5-S6.csv'

dump_data = pd.read_csv(dump_name)
adata = sc.AnnData(X=dump_data[list(set(sum(dump_features + ILC_NK_features, [])))], 
                            obs=dump_data[['Major_cell_type', 'Cell_type']],
                            obsm={'X_original_umap': dump_data[['Original_UMAP1', 'Original_UMAP2']].values, 
                                  'X_scmer_umap': dump_data[['SCMER_UMAP1', 'SCMER_UMAP2']].values})

sc.settings.set_figure_params(dpi=medium_dpi, facecolor='white')
sc.pl.embedding(adata, "original_umap", color=['Major_cell_type', 'Cell_type'], title=['Original', 'Original'])
sc.pl.embedding(adata, "scmer_umap", color=['Major_cell_type', 'Cell_type'], title=['SCMER', 'SCMER'])

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')
for row in dump_features:
    sc.pl.embedding(adata, "original_umap", color=row, color_map="inferno")

for row in dump_features:
    sc.pl.embedding(adata, "scmer_umap", color=row, color_map="inferno")
    
sc.settings.set_figure_params(dpi=medium_dpi, facecolor='white')
sc.pl.stacked_violin(adata, ['CD79A', 'LYZ', 'FCER2', 'ANKRD28', 'JUN', 'FOS', 'CD69', 'ODF2L', 'DUSP1'], groupby="Major_cell_type")

In [ ]:
NK_score = adata[:, ['CD160', 'CCL4', 'GZMK', 'GZMA', 'IRF8']].X
NK_score = (NK_score / NK_score.std(axis=0)).max(axis=1)

ILC1_score = adata[:, ['SELL', 'GNLY', 'IL7R']].X
ILC1_score = (ILC1_score / ILC1_score.std(axis=0)).max(axis=1)

adata.obs['NK_vs_ILC1'] = ['Other' if j != 'Group1 ILC' else 'NK' if i > 0 else 'ILC1' if i < 0 else '?' 
                           for i, j in zip((NK_score - ILC1_score), adata.obs['Cell_type'])]

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')

sc.pl.embedding(adata[(0 < adata.obsm['X_original_umap'][:, 0]) & 
                      (adata.obsm['X_original_umap'][:, 0] < 5) &
                      (5 < adata.obsm['X_original_umap'][:, 1]) &
                      (adata.obsm['X_original_umap'][:, 1] < 10), :], 
           "original_umap", color=['NK_vs_ILC1'], groups=['NK', 'ILC1'], 
           legend_fontsize=15, ncols=4, size=100.)
                 
sc.pl.embedding(adata[(-2 < adata.obsm['X_scmer_umap'][:, 0]) & 
                      (adata.obsm['X_scmer_umap'][:, 0] < 2) &
                      (9.5 < adata.obsm['X_scmer_umap'][:, 1]) &
                      (adata.obsm['X_scmer_umap'][:, 1] < 13.5), :], 
           "scmer_umap", color=['NK_vs_ILC1'], groups=['NK', 'ILC1'], 
           legend_fontsize=15, ncols=4, size=100.)

temp = adata[(0 < adata.obsm['X_original_umap'][:, 0]) & 
            (adata.obsm['X_original_umap'][:, 0] < 5) &
            (5 < adata.obsm['X_original_umap'][:, 1]) &
            (adata.obsm['X_original_umap'][:, 1] < 10), :]
for row in ILC_NK_features:
    sc.pl.embedding(temp, "original_umap", color=row, color_map="inferno")
    
temp = adata[(-2 < adata.obsm['X_scmer_umap'][:, 0]) & 
                  (adata.obsm['X_scmer_umap'][:, 0] < 2) &
                  (9.5 < adata.obsm['X_scmer_umap'][:, 1]) &
                  (adata.obsm['X_scmer_umap'][:, 1] < 13.5), :]
for row in ILC_NK_features:
    sc.pl.embedding(temp, "scmer_umap", color=row, color_map="inferno")

## Fig. 4 + Supplementary Figure 7 + Table 3

**Table 3** is generated by taking the intersection of `SCMER-feature-weights/bm-weigths.csv` and `Gene-sets/bm-gene-sets.csv`.

In [ ]:
dump_features = [['GPR183', 'PRTN3', 'PDZD8'], ['CLC', 'PRG2'], ['AHSP', 'CA1', 'VPREB1']]
dump_data = pd.read_csv('Fig-4-S7.csv')
adata = sc.AnnData(X=dump_data[list(set(sum(dump_features, [])))], 
                            obs=dump_data[['Cell_type']],
                            obsm={'X_original_umap': dump_data[['Original_UMAP1', 'Original_UMAP2']].values, 
                                  'X_scmer_umap': dump_data[['SCMER_UMAP1', 'SCMER_UMAP2']].values})

sc.settings.set_figure_params(dpi=medium_dpi, facecolor='white')
sc.pl.embedding(adata, "original_umap", color=['Cell_type'], title="Original")
sc.pl.embedding(adata, "scmer_umap", color=['Cell_type'], title="SCMER")

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')
for row in dump_features:
    sc.pl.embedding(adata, "original_umap", color=row, color_map="inferno")

for row in dump_features:
    sc.pl.embedding(adata, "scmer_umap", color=row, color_map="inferno")

In [ ]:
selected_features = set(pd.read_csv("SCMER-feature-weights/bm-weigths.csv", index_col=0).index.tolist())
established_features = pd.read_csv("Gene-sets/bm-gene-sets.csv", index_col=0)

df = established_features.copy()
for i in df.index:
    df.loc[i] = ",".join(set(df.loc[i, 'Genes'].split(", ")).intersection(selected_features))
    
df.style

## Fig. 5 + Supplementary Figure 8 and 9

**Fig. 5j** and **Supplementary Figure 8a** are generated in R. Please see folder `A549-heatmap`.

In [ ]:
dump_data = pd.read_csv('Fig-5abdef.csv', index_col=0)

sns.jointplot(data=dump_data.sort_values(by='Treatment_time', ascending=False), x="Original_UMAP1", y="Original_UMAP2", 
              hue="Treatment_time", s=5,
             palette=[sns.color_palette("tab10")[i] for i in [2, 0, 1]])

In [ ]:
sns.jointplot(data=dump_data.sort_values(by='Treatment_time', ascending=False), x="SCMER_UMAP1", y="SCMER_UMAP2", 
              hue="Treatment_time", s=5,
             palette=[sns.color_palette("tab10")[i] for i in [2, 0, 1]])

In [ ]:
df = pd.read_csv('Fig-5c.csv')

sns.jointplot(data=df.sort_values(by='treatment_time', ascending=False), x="UMAP1", y="UMAP2", hue="treatment_time", s=5,
             palette=[sns.color_palette("tab10")[i] for i in [2, 0, 1]])

In [ ]:
adata = sc.AnnData(X=dump_data.iloc[:, 5:], 
                    obs=dump_data[['Treatment_time']],
                    obsm={'X_original_umap': dump_data[['Original_UMAP1', 'Original_UMAP2']].values, 
                          'X_scmer_umap': dump_data[['SCMER_UMAP1', 'SCMER_UMAP2']].values,
                         })

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')
sc.pl.embedding(adata, "original_umap", color=adata.var_names, color_map="inferno", ncols=6)
sc.pl.embedding(adata, "scmer_umap", color=adata.var_names, color_map="inferno", ncols=6)

In [ ]:
dump_data = pd.read_csv('Fig-5ghi.csv', index_col=0)
adata = sc.AnnData(X=dump_data.iloc[:, 2:],
                    obsm={'X_peak_umap': dump_data[['Peak_UMAP1', 'Peak_UMAP2']].values})

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')
sc.pl.embedding(adata, "peak_umap", color=adata.var_names, color_map="inferno", ncols=6)

## Fig. 6 + Supplementary Figure 10

In [ ]:
dump_data = pd.read_csv('Fig-6-S10.csv')

adata = sc.AnnData(X=dump_data.iloc[:, 10:], 
                            obs=dump_data[['celltype.oi', 'celltype.l1.5']],
                            obsm={'X_protein_umap': dump_data[['Protein_UMAP1', 'Protein_UMAP2']].values, 
                                  'X_rna_umap': dump_data[['RNA_UMAP1', 'RNA_UMAP2']].values, 
                                  'X_coding_umap': dump_data[['Coding_UMAP1', 'Coding_UMAP2']].values, 
                                  'X_scmer_umap': dump_data[['SCMER_UMAP1', 'SCMER_UMAP2']].values})

In [ ]:
celltype_oi = sorted(list(set(adata.obs['celltype.oi'].unique().tolist()) - {'zzz'}))
sc.pl.embedding(adata, 'rna_umap', color=['celltype.oi'], groups=celltype_oi, legend_fontsize=8., size=4., title="RNA")
sc.pl.embedding(adata, 'protein_umap', color=['celltype.oi'], groups=celltype_oi, legend_fontsize=8., size=4., title="Protein")
sc.pl.embedding(adata, 'scmer_umap', color=['celltype.oi'], groups=celltype_oi, legend_fontsize=8., size=4., title="SCMER")
sc.pl.embedding(adata, 'coding_umap', color=['celltype.oi'], groups=celltype_oi, legend_fontsize=8., size=4., 
                title="Genes Coding the measured proteins")

sc.settings.set_figure_params(dpi=medium_dpi, facecolor='white')

sc.pl.embedding(adata, 'rna_umap', color=['celltype.l1.5'], legend_fontsize=8., size=4., title="RNA")
sc.pl.embedding(adata, 'protein_umap', color=['celltype.l1.5'], legend_fontsize=8., size=4., title="Protein")
sc.pl.embedding(adata, 'scmer_umap', color=['celltype.l1.5'], legend_fontsize=8., size=4., title="SCMER")
sc.pl.embedding(adata, 'coding_umap', color=['celltype.l1.5'], legend_fontsize=8., size=4., title="Genes Coding the measured proteins")

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')

sc.pl.embedding(adata, 'protein_umap', color=['CD4', 'CD56', 'CD45RA', 'CD45RO'], 
                legend_fontsize=8., size=4., color_map="inferno")
sc.pl.embedding(adata, 'protein_umap', color=['RNA_CD4', 'RNA_NCAM1', 'RNA_PTPRC'], 
                legend_fontsize=8., size=4., color_map="inferno")
sc.pl.embedding(adata, 'protein_umap', color=['RNA_GPR183', 'RNA_KLRF1', 'RNA_CD79B', 'RNA_S100A4'], 
                legend_fontsize=8., size=4., color_map="inferno")

In [ ]:
dump_data = pd.read_csv('Fig-S10efg.csv')

adata = sc.AnnData(X=None, 
                    obs=dump_data[['celltype.oi', 'celltype.l1.5']],
                    obsm={'X_protein_umap': dump_data[['Protein_UMAP1', 'Protein_UMAP2']].values, 
                          'X_rna_umap': dump_data[['RNA_UMAP1', 'RNA_UMAP2']].values,
                          'X_scmer_umap': dump_data[['SCMER_UMAP1', 'SCMER_UMAP2']].values})

sc.settings.set_figure_params(dpi=medium_dpi, facecolor='white')

sc.pl.embedding(adata, 'rna_umap', color=['celltype.l1.5'], legend_fontsize=8., size=4., title="RNA")
sc.pl.embedding(adata, 'protein_umap', color=['celltype.l1.5'], legend_fontsize=8., size=4., title="Protein")
sc.pl.embedding(adata, 'scmer_umap', color=['celltype.l1.5'], legend_fontsize=8., size=4., title="SCMER")

## Supplementary Figure 11

In [ ]:
with open("melanoma-cmp.json", 'r') as f:
    locals().update(json.load(f))
    
sc.settings.set_figure_params(dpi=large_dpi, facecolor='white')

fig = plt.figure(figsize=(5, 3))
plt.plot(scmer_x, scmer_rec, lw=1.5, color='C0')
plt.plot(x, rand_avg, lw=1., color='C1')
fig.axes[0].fill_between(x, np.maximum(0, np.array(rand_avg) - 1.645 * np.array(rand_sd)), 
                         np.minimum(11, np.array(rand_avg) + 1.645 * np.array(rand_sd)), 
                         alpha=.1, facecolor='C1')

plt.plot(x, heg_rec, lw=1., color='C2', alpha=.8)
plt.plot(x, hvg_rec, lw=1., color='C3', alpha=.8)
plt.plot(deg_x, deg_rec, linestyle='--', lw=1., color='C4', alpha=.8)
plt.plot(scmarker_x, scmarker_rec, lw=1., color='C5', alpha=.8)

plt.plot(monocle_x, monocle_rec, linestyle='--', lw=1., color='C6', alpha=.8)
plt.plot(monocle_unsup_x, monocle_unsup_rec, lw=1., color='C6', alpha=.8)

plt.plot(rankcorr_x, rankcorr_rec, lw=1., linestyle='--', color='C7', alpha=.8)
plt.plot(rankcorr_unsup_x, rankcorr_unsup_rec, lw=1., color='C7', alpha=.8)
plt.hlines(11, 0, 2000, colors='gray', lw=1., linestyles='dashed')
plt.grid(b=None)

fig.axes[0].spines['right'].set_visible(False)
fig.axes[0].spines['top'].set_visible(False)

plt.legend(['SCMER', 'Random', 'HXG', 'HVG', 'Supervised DEG', 'SCMarker', 'Supervised Monocle', 'Unspervised Monocle', 
            'Supervised RankCorr', 'Unsupervised RankCorr'], 
           loc='lower right', ncol=2, prop={'size': 10}, handlelength=1.)

plt.plot(scmer_x, scmer_rec, lw=1.5)
plt.ylim([-1, 11 + .5])
plt.xlim([0, 1000])
plt.yticks([0, 5, 10, 11])

plt.ylabel('Number of\nrecalled gene sets')
plt.xlabel('Number of selected markers')
plt.title('Supplementary Figure 11a')


fig = plt.figure(figsize=(5, 3))
plt.plot(scmer_x, scmer_rec, lw=1.5, color='C0')
plt.plot(x, rand_avg, lw=1., color='C1')
fig.axes[0].fill_between(x, np.maximum(0, np.array(rand_avg) - 1.645 * np.array(rand_sd)), 
                         np.minimum(11, np.array(rand_avg) + 1.645 * np.array(rand_sd)), 
                         alpha=.1, facecolor='C1')

plt.plot(x, heg_rec, lw=1., color='C2', alpha=.8)
plt.plot(x, hvg_rec, lw=1., color='C3', alpha=.8)
plt.plot(deg_x, deg_rec, lw=1., color='C4', alpha=.8)
plt.plot(scmarker_x, scmarker_rec, lw=1., color='C5', alpha=.8)

plt.plot(monocle_x, monocle_rec, lw=1., color='C6', alpha=.8)

plt.plot(rankcorr_unsup_x, rankcorr_unsup_rec, lw=1., color='C7', alpha=.8)

plt.hlines(11, 0, 2000, colors='gray', lw=1., linestyles='dashed')
plt.grid(b=None)

fig.axes[0].spines['right'].set_visible(False)
fig.axes[0].spines['top'].set_visible(False)

plt.legend(['SCMER', 'Random', 'HXG', 'HVG', 'DEG', 'SCMarker', 'Monocle', 
            'RankCorr'], 
           loc='lower right', ncol=2, prop={'size': 15}, handlelength=1.)

plt.plot(scmer_x, scmer_rec, lw=1.5)
plt.ylim([-1, 11 + .5])
plt.xlim([0, 1000])
plt.yticks([0, 5, 10, 11])

plt.ylabel('Number of\nrecalled gene sets')
plt.xlabel('Number of selected markers')
plt.title('Fig. 2c')

In [ ]:
with open("ileum-cmp.json", 'r') as f:
    locals().update(json.load(f))

sc.settings.set_figure_params(dpi=large_dpi, facecolor='white')

fig = plt.figure(figsize=(5, 3))
plt.plot(scmer_x, scmer_rec, lw=1.5, color='C0')
plt.plot(x[:-2], rand_avg, lw=1., color='C1')
fig.axes[0].fill_between(x[:-2], np.maximum(0, np.array(rand_avg) - 1.645 * np.array(rand_sd)), 
                         np.minimum(12, np.array(rand_avg) + 1.645 * np.array(rand_sd)), 
                         alpha=.1, facecolor='C1')

plt.plot(x, heg_rec, lw=1., color='C2', alpha=.8)
plt.plot(x, hvg_rec, lw=1., color='C3', alpha=.8)
plt.plot(deg_x, deg_rec, linestyle='--', lw=1., color='C4', alpha=.8)
plt.plot(scmarker_x, scmarker_rec, lw=1., color='C5', alpha=.8)

plt.plot(monocle_x, monocle_rec, linestyle='--', lw=1., color='C6', alpha=.8)
plt.plot(rankcorr_x, rankcorr_rec, lw=1., linestyle='--', color='C7', alpha=.8)
plt.plot(rankcorr_unsup_x, rankcorr_unsup_rec, lw=1., color='C7', alpha=.8)
plt.hlines(27, 0, 5000, colors='gray', lw=1., linestyles='dashed')
plt.hlines(31, 0, 2000, colors='gray', lw=1., linestyles='dashed')
plt.grid(b=None)

fig.axes[0].spines['right'].set_visible(False)
fig.axes[0].spines['top'].set_visible(False)

plt.legend(['SCMER', 'Random', 'HXG', 'HVG', 'Supervised DEG', 'SCMarker', 'Supervised Monocle', 
            'Supervised RankCorr', 'Unsupervised RankCorr'], 
           loc='lower right', ncol=2, prop={'size': 9}, handlelength=1.)

plt.plot(scmer_x, scmer_rec, lw=1.5)
plt.ylim([-1, 12 + .5])
plt.xlim([0, 1000])
plt.yticks([0, 5, 10, 31])

plt.ylabel('Number of\nrecalled gene sets')
plt.xlabel('Number of selected markers')
plt.title('Supplementary Figure 11b')


fig = plt.figure(figsize=(5, 3))
plt.plot(scmer_x, scmer_rec, lw=1.5, color='C0')
plt.plot(x[:-2], rand_avg, lw=1., color='C1')
fig.axes[0].fill_between(x[:-2], np.maximum(0, np.array(rand_avg) - 1.645 * np.array(rand_sd)), 
                         np.minimum(12, np.array(rand_avg) + 1.645 * np.array(rand_sd)), 
                         alpha=.1, facecolor='C1')

plt.plot(x, heg_rec, lw=1., color='C2', alpha=.8)
plt.plot(x, hvg_rec, lw=1., color='C3', alpha=.8)
plt.plot(deg_x, deg_rec, lw=1., color='C4', alpha=.8)
plt.plot(scmarker_x, scmarker_rec, lw=1., color='C5', alpha=.8)

plt.plot(monocle_x, monocle_rec, lw=1., color='C6', alpha=.8)

plt.plot(rankcorr_unsup_x, rankcorr_unsup_rec, lw=1., color='C7', alpha=.8)
plt.hlines(27, 0, 5000, colors='gray', lw=1., linestyles='dashed')
plt.hlines(31, 0, 2000, colors='gray', lw=1., linestyles='dashed')
plt.grid(b=None)

fig.axes[0].spines['right'].set_visible(False)
fig.axes[0].spines['top'].set_visible(False)

plt.legend(['SCMER', 'Random', 'HXG', 'HVG', 'DEG', 'SCMarker', 'Monocle', 
            'RankCorr'], 
           loc='lower right', ncol=2, prop={'size': 12}, handlelength=1.)

plt.plot(scmer_x, scmer_rec, lw=1.5)
plt.ylim([-1, 12 + .5])
plt.xlim([0, 1000])
plt.yticks([0, 5, 10, 31])

plt.ylabel('Number of\nrecalled gene sets')
plt.xlabel('Number of selected markers')
plt.title('Fig. 3i')

In [ ]:
with open("bone-marrow-cmp.json", 'r') as f:
    locals().update(json.load(f))
    
sc.settings.set_figure_params(dpi=large_dpi, facecolor='white')

fig = plt.figure(figsize=(5, 3))
plt.plot(scmer_x, scmer_rec, lw=1.5, color='C0')
plt.plot(x, rand_avg, lw=1., color='C1')
fig.axes[0].fill_between(x, np.maximum(0, np.array(rand_avg) - 1.645 * np.array(rand_sd)), 
                         np.minimum(12, np.array(rand_avg) + 1.645 * np.array(rand_sd)), 
                         alpha=.1, facecolor='C1')

plt.plot(x, heg_rec, lw=1., color='C2', alpha=.8)
plt.plot(x, hvg_rec, lw=1., color='C3', alpha=.8)
plt.plot(deg_x, deg_rec, linestyle='--', lw=1., color='C4', alpha=.8)
plt.plot(scmarker_x, scmarker_rec, lw=1., color='C5', alpha=.8)

plt.plot(monocle_x, monocle_rec, linestyle='--', lw=1., color='C6', alpha=.8)
plt.plot(rankcorr_x, rankcorr_rec, lw=1., linestyle='--', color='C7', alpha=.8)
plt.plot(rankcorr_unsup_x, rankcorr_unsup_rec, lw=1., color='C7', alpha=.8)
plt.hlines(12, 0, 2000, colors='gray', lw=1., linestyles='dashed')
plt.grid(b=None)

fig.axes[0].spines['right'].set_visible(False)
fig.axes[0].spines['top'].set_visible(False)

plt.legend(['SCMER', 'Random', 'HXG', 'HVG', 'Supervised DEG', 'SCMarker', 'Supervised Monocle', 
            'Supervised RankCorr', 'Unsupervised RankCorr'], 
           loc='lower right', ncol=2, prop={'size': 9}, handlelength=1.)

plt.plot(scmer_x, scmer_rec, lw=1.5)
plt.ylim([-1, 12 + .5])
plt.xlim([0, 1000])
plt.yticks([0, 5, 10, 12])

plt.ylabel('Number of\nrecalled gene sets')
plt.xlabel('Number of selected markers')
plt.title('Supplementary Figure 11c')


fig = plt.figure(figsize=(5, 3))
plt.plot(scmer_x, scmer_rec, lw=1.5, color='C0')
plt.plot(x, rand_avg, lw=1., color='C1')
fig.axes[0].fill_between(x, np.maximum(0, np.array(rand_avg) - 1.645 * np.array(rand_sd)), 
                         np.minimum(12, np.array(rand_avg) + 1.645 * np.array(rand_sd)), 
                         alpha=.1, facecolor='C1')

plt.plot(x, heg_rec, lw=1., color='C2', alpha=.8)
plt.plot(x, hvg_rec, lw=1., color='C3', alpha=.8)
plt.plot(deg_x, deg_rec, lw=1., color='C4', alpha=.8)
plt.plot(scmarker_x, scmarker_rec, lw=1., color='C5', alpha=.8)

plt.plot(monocle_x, monocle_rec, lw=1., color='C6', alpha=.8)

plt.plot(rankcorr_unsup_x, rankcorr_unsup_rec, lw=1., color='C7', alpha=.8)

plt.hlines(12, 0, 2000, colors='gray', lw=1., linestyles='dashed')
plt.grid(b=None)

fig.axes[0].spines['right'].set_visible(False)
fig.axes[0].spines['top'].set_visible(False)

plt.legend(['SCMER', 'Random', 'HXG', 'HVG', 'DEG', 'SCMarker', 'Monocle', 
            'RankCorr'], 
           loc='lower right', ncol=2, prop={'size': 12}, handlelength=1.)

plt.plot(scmer_x, scmer_rec, lw=1.5)
plt.ylim([-1, 12 + .5])
plt.xlim([0, 1000])
plt.yticks([0, 5, 10, 12])

plt.ylabel('Number of\nrecalled gene sets')
plt.xlabel('Number of selected markers')
plt.title('Fig. 4c')

## Supplementary Figure 12, 13, 14, and 15

In [ ]:
dump_data = pd.read_csv('Fig-S12ab-S13abcde-S14.csv', index_col=0)
dump_data.columns

In [ ]:
adata = sc.AnnData(X=dump_data.iloc[:, 13:-3], 
                    obs=dump_data[['Cell type']],
                    obsm={'X_protein_umap': dump_data[['Protein_UMAP1', 'Protein_UMAP2']].values, 
                          'X_rna_umap': dump_data[['RNA_UMAP1', 'RNA_UMAP2']].values,
                          'X_scmer_umap': dump_data[['SCMER_UMAP1', 'SCMER_UMAP2']].values,
                          'X_scmer26_umap': dump_data[['SCMER26_UMAP1', 'SCMER26_UMAP2']].values,
                          'X_scmer_pre_umap': dump_data[['SCMER_Preselect_UMAP1', 'SCMER_Preselect_UMAP2']].values,
                         'X_coding_umap': dump_data[['Coding_UMAP1', 'Coding_UMAP2']].values,
                         'X_cor10_umap': dump_data[['Cor10_UMAP1', 'Cor10_UMAP2']].values,
                         'X_cor19_umap': dump_data[['Cor19_UMAP1', 'Cor19_UMAP2']].values})

sc.settings.set_figure_params(dpi=medium_dpi, facecolor='white')

sc.pl.embedding(adata, 'protein_umap', color=['Cell type'], legend_fontsize=8., size=4., title='Protein')
sc.pl.embedding(adata, 'scmer_umap', color=['Cell type'], legend_fontsize=8., size=4., title='SCMER')

sc.pl.embedding(adata, 'rna_umap', color=['Cell type'], legend_fontsize=8., size=4., title='RNA')
sc.pl.embedding(adata, 'coding_umap', color=['Cell type'], legend_fontsize=8., size=4., title='14 Genes coding the proteins')
sc.pl.embedding(adata, 'cor10_umap', color=['Cell type'], legend_fontsize=8., size=4., title='Top-10 Correlated')
sc.pl.embedding(adata, 'cor19_umap', color=['Cell type'], legend_fontsize=8., size=4., title='Top-19 Correlated')
sc.pl.embedding(adata, 'scmer26_umap', color=['Cell type'], legend_fontsize=8., size=4., title='SCMER 26 Features')
sc.pl.embedding(adata, 'scmer_pre_umap', color=['Cell type'], legend_fontsize=8., size=4., title='SCMER 14 Preselected Coding + 26 Features')

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')
for k, v in {'PROTEIN_CD2': ['CD2'],
               'PROTEIN_CD3': ['CD3D', 'CD3E', 'CD3G'],
               'PROTEIN_CD4': ['CD4'],
               'PROTEIN_CD8': ['CD8A', 'CD8B'],
               'PROTEIN_CD11c': ['ITGAX'],
               'PROTEIN_CD14': ['CD14'],
               'PROTEIN_CD16': ['FCGR3A', 'FCGR3B'],
               'PROTEIN_CD19': ['CD19'],
               'PROTEIN_CD45RA': ['PTPRC'],
               'PROTEIN_CD57': ['B3GAT1']
              }.items():
    sc.pl.embedding(adata, 'protein_umap', color=[k] + v, legend_fontsize=8., size=4., color_map="inferno")

In [ ]:
sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')

corrMatrix = pd.read_csv("Fig-S12c.csv", index_col=0)
fig, ax = plt.subplots(figsize=(5, 3))
sns.heatmap(ax=ax, data=corrMatrix.abs(), xticklabels=True, yticklabels=True, vmin=0., vmax=1.)
plt.show()

corrMatrix = pd.read_csv("Fig-S12d.csv", index_col=0)
fig, ax = plt.subplots(figsize=(5, 3))
sns.heatmap(ax=ax, data=corrMatrix.abs(), xticklabels=True, yticklabels=True, vmin=0., vmax=1.)
plt.show()

corrMatrix = pd.read_csv("Fig-S13f.csv", index_col=0)
fig, ax = plt.subplots(figsize=(8, 3))
sns.heatmap(ax=ax, data=corrMatrix.abs(), xticklabels=True, yticklabels=True, vmin=0., vmax=1.)
plt.show()

corrMatrix = pd.read_csv("Fig-S15a.csv", index_col=0)
fig, ax = plt.subplots(figsize=(13, 3))
sns.heatmap(ax=ax, data=corrMatrix.abs(), xticklabels=True, yticklabels=True, vmin=0., vmax=1.)
plt.show()

In [ ]:
dump_data = pd.read_csv('Fig-S15c.csv')
adata = sc.AnnData(X=None, 
                    obs=dump_data[['Cell_type']],
                    obsm={'X_scmer_novel_umap': dump_data[['SCMER_Novel_UMAP1', 'SCMER_Novel_UMAP2']].values})

sc.pl.embedding(adata, 'scmer_novel_umap', color=['Cell_type'], legend_loc="on data", size=4., title='SCMER Novel Features')

## Supplementary Figure 16

In [ ]:
sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')
pd.read_csv('Fig-S16c.csv', index_col=0)['weight'].plot(kind='barh', figsize=(3, 8))

In [ ]:
dump_data = pd.read_csv('Fig-S16abd.csv')
adata = sc.AnnData(X=None, 
                    obs=dump_data[['Cell_type']],
                    obsm={'X_original_umap': dump_data[['Original_UMAP1', 'Original_UMAP2']].values, 
                          'X_scmer_umap': dump_data[['SCMER_UMAP1', 'SCMER_UMAP2']].values,
                          'X_random1_umap': dump_data[['Random1_UMAP1', 'Random1_UMAP2']].values,
                          'X_random2_umap': dump_data[['Random2_UMAP1', 'Random2_UMAP2']].values,
                          'X_random3_umap': dump_data[['Random3_UMAP1', 'Random3_UMAP2']].values,
                          'X_random4_umap': dump_data[['Random4_UMAP1', 'Random4_UMAP2']].values,
                          'X_random5_umap': dump_data[['Random5_UMAP1', 'Random5_UMAP2']].values,
                         })

sc.settings.set_figure_params(dpi=small_dpi, facecolor='white')

sc.pl.embedding(adata, "original_umap", color=['Cell_type'], title='Original')
sc.pl.embedding(adata, "scmer_umap", color=['Cell_type'], title='SCMER')

for i in range(5):
    sc.pl.embedding(adata, "random" + str(i + 1) + '_umap', color=['Cell_type'], title='Random')